# Python 知识

# 代码分析

## class IndexingBase
```python
IndexingBaseT = tp.TypeVar("IndexingBaseT", bound="IndexingBase")

class IndexingBase:
    def indexing_func(self: IndexingBaseT, pd_indexing_func: tp.Callable, **kwargs) -> IndexingBaseT:
        raise NotImplementedError
```

## class LocBase
```python
class LocBase:
    def __init__(self, indexing_func: tp.Callable, **kwargs) -> None:
        self._indexing_func = indexing_func  
        self._indexing_kwargs = kwargs  

    @property
    def indexing_func(self) -> tp.Callable:
        return self._indexing_func

    @property
    def indexing_kwargs(self) -> dict:
        return self._indexing_kwargs

    def __getitem__(self, key: tp.Any) -> tp.Any:
        raise NotImplementedError
```

## class iLoc(LocBase)
```python
class iLoc(LocBase):
    def __getitem__(self, key: tp.Any) -> tp.Any:
        return self.indexing_func(lambda x: x.iloc.__getitem__(key), **self.indexing_kwargs)
```

## class Loc(LocBase)
```python
class Loc(LocBase):
    def __getitem__(self, key: tp.Any) -> tp.Any:
        return self.indexing_func(lambda x: x.loc.__getitem__(key), **self.indexing_kwargs)
```

## class PandasIndexer(IndexingBase)
```python
PandasIndexerT = tp.TypeVar("PandasIndexerT", bound="PandasIndexer")

class PandasIndexer(IndexingBase):
    def __init__(self, **kwargs) -> None:
        self._iloc = iLoc(self.indexing_func, **kwargs)
        self._loc = Loc(self.indexing_func, **kwargs)
        self._indexing_kwargs = kwargs

    @property
    def indexing_kwargs(self) -> dict:
        return self._indexing_kwargs

    @property
    def iloc(self) -> iLoc:
        return self._iloc

    iloc.__doc__ = iLoc.__doc__

    @property
    def loc(self) -> Loc:
        return self._loc

    loc.__doc__ = Loc.__doc__

    def xs(self: PandasIndexerT, *args, **kwargs) -> PandasIndexerT:
        return self.indexing_func(lambda x: x.xs(*args, **kwargs), **self.indexing_kwargs)

    def __getitem__(self: PandasIndexerT, key: tp.Any) -> PandasIndexerT:
        return self.indexing_func(lambda x: x.__getitem__(key), **self.indexing_kwargs)
```

## class ParamLoc(LocBase)
```python
class ParamLoc(LocBase):
    def __init__(self, mapper: tp.Series, indexing_func: tp.Callable, level_name: tp.Level = None, **kwargs) -> None:
        checks.assert_instance_of(mapper, pd.Series)

        if mapper.dtype == 'O':  # 'O'表示object类型
            mapper = mapper.astype(str)
            
        self._mapper = mapper
        self._level_name = level_name

        LocBase.__init__(self, indexing_func, **kwargs)

    @property
    def mapper(self) -> tp.Series:
        return self._mapper

    @property
    def level_name(self) -> tp.Level:
        return self._level_name

    def get_indices(self, key: tp.Any) -> tp.Array1d:
        if self.mapper.dtype == 'O':
            if isinstance(key, slice):
                # 处理切片对象，转换start和stop为字符串
                start = str(key.start) if key.start is not None else None
                stop = str(key.stop) if key.stop is not None else None
                key = slice(start, stop, key.step)
            elif isinstance(key, (list, np.ndarray)):
                key = list(map(str, key))
            else:
                key = str(key)
                
        mapper = pd.Series(np.arange(len(self.mapper.index)), index=self.mapper.values)
        
        indices = mapper.loc.__getitem__(key)
        
        if isinstance(indices, pd.Series):
            indices = indices.values
            
        return indices

    def __getitem__(self, key: tp.Any) -> tp.Any:
        indices = self.get_indices(key)
        
        is_multiple = isinstance(key, (slice, list, np.ndarray))

        def pd_indexing_func(obj: tp.SeriesFrame) -> tp.MaybeSeriesFrame:
            new_obj = obj.iloc[:, indices]
            if not is_multiple:
                if self.level_name is not None:
                    if checks.is_frame(new_obj):
                        if isinstance(new_obj.columns, pd.MultiIndex):
                            new_obj.columns = index_fns.drop_levels(new_obj.columns, self.level_name)
                            
            return new_obj

        return self.indexing_func(pd_indexing_func, **self.indexing_kwargs)
```

## def indexing_on_mapper
```python
def indexing_on_mapper(mapper: tp.Series, ref_obj: tp.SeriesFrame,
                       pd_indexing_func: tp.Callable) -> tp.Optional[tp.Series]:

    checks.assert_instance_of(mapper, pd.Series)
    checks.assert_instance_of(ref_obj, (pd.Series, pd.DataFrame))

    df_range_mapper = reshape_fns.broadcast_to(np.arange(len(mapper.index)), ref_obj)
    
    loced_range_mapper = pd_indexing_func(df_range_mapper)
    
    new_mapper = mapper.iloc[loced_range_mapper.values[0]]
    
    if checks.is_frame(loced_range_mapper):
        return pd.Series(new_mapper.values, index=loced_range_mapper.columns, name=mapper.name)
    elif checks.is_series(loced_range_mapper):
        return pd.Series([new_mapper], index=[loced_range_mapper.name], name=mapper.name)
    
    return None
```

## def build_param_indexer
```python
def build_param_indexer(param_names: tp.Sequence[str], class_name: str = 'ParamIndexer',
                        module_name: tp.Optional[str] = None) -> tp.Type[IndexingBase]:
    
    class ParamIndexer(IndexingBase):
        def __init__(self, param_mappers: tp.Sequence[tp.Series],
                     level_names: tp.Optional[tp.LevelSequence] = None, **kwargs) -> None:

            checks.assert_len_equal(param_names, param_mappers)
            # 为每个参数创建对应的ParamLoc索引器
            for i, param_name in enumerate(param_names):
                level_name = level_names[i] if level_names is not None else None
                _param_loc = ParamLoc(param_mappers[i], self.indexing_func, level_name=level_name, **kwargs)
                # 将ParamLoc实例设置为私有属性，命名格式为_{param_name}_loc
                setattr(self, f'_{param_name}_loc', _param_loc)

    for i, param_name in enumerate(param_names):
        
        def param_loc(self, _param_name=param_name) -> ParamLoc:
            return getattr(self, f'_{_param_name}_loc')

        # 为属性方法设置文档字符串
        param_loc.__doc__ = f"""Access a group of columns by parameter `{param_name}` using `pd.Series.loc`.
        
        Forwards this operation to each Series/DataFrame and returns a new class instance.
        """

        setattr(ParamIndexer, param_name + '_loc', property(param_loc))

    ParamIndexer.__name__ = class_name  
    ParamIndexer.__qualname__ = class_name  
    if module_name is not None:
        ParamIndexer.__module__ = module_name  

    return ParamIndexer
```